In [1]:
import yt
from matplotlib import rc_context
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
print(yt.__version__)

4.4.0


In [3]:
Norm = np.array([1.0,
  1./3.,
  1./3.,
  1./3.,
  2./3.,
  4./3.,
  4./9.,
  1./9.,
  1./9.,
  1./9.,
  2./3.,
  2./3.,
  2./3.,
  2./9.,
  2./9.,
  2./9.,
  2.0,
  4./3.,
  4./9.])
print(Norm.shape)

(19,)


In [4]:
############    modify if needed    ############################
step1 = 2000#702000
step2 = 400000#900000
plot_int = 2000
temperature = 1e-5
alpha0 = 0.
cs2 = 1./3.
nx = 16#32
ny = 16#32
nz = 16#32
ncomp = 19  # nvel in LBM
###############################################################

Nframes = (step2 - step1)//plot_int + 1
fnoise_ndarray = np.zeros((Nframes, ncomp, nx, ny, nz))
gnoise_ndarray = np.zeros((Nframes, ncomp, nx, ny, nz))

plot_file_fnoise_root = "./data_mixture/lbm_data_shshan_alpha0_" + "{:.2f}".format(alpha0) +\
          "_xi_" + "{:.1e}".format(temperature) + "_size{:d}-{:d}-{:d}_continue/data_fnoise/fn".format(nx,ny,nz)
plot_file_gnoise_root = "./data_mixture/lbm_data_shshan_alpha0_" + "{:.2f}".format(alpha0) +\
          "_xi_" + "{:.1e}".format(temperature) + "_size{:d}-{:d}-{:d}_continue/data_gnoise/gn".format(nx,ny,nz)

'''
plot_file_fnoise = plot_file_fnoise_root + f"{step1:06d}"
ds0 = yt.load(plot_file_fnoise)
print(ds0.field_list)
'''

# load in noise multifab of each frame into ndarray
for n in range(step1, step2+1, plot_int):
    plot_file_fnoise = plot_file_fnoise_root + f"{n:07d}"
    plot_file_gnoise = plot_file_gnoise_root + f"{n:07d}"
    ds0 = yt.load(plot_file_fnoise)
    fnoise_data_frame = ds0.covering_grid(
    level=0, left_edge=ds0.domain_left_edge, dims=ds0.domain_dimensions)
    ds1 = yt.load(plot_file_gnoise)
    gnoise_data_frame = ds1.covering_grid(
    level=0, left_edge=ds1.domain_left_edge, dims=ds1.domain_dimensions)
    for a in range(0, ncomp):
        fnoise_ndarray[(n-step1)//plot_int, a, :,:,:] = fnoise_data_frame["boxlib", "fa{:d}".format(a)]
        gnoise_ndarray[(n-step1)//plot_int, a, :,:,:] = gnoise_data_frame["boxlib", "ga{:d}".format(a)]
    

print(fnoise_ndarray.shape)


yt : [INFO     ] 2025-04-02 16:23:00,187 Parameters: current_time              = 2000.0
yt : [INFO     ] 2025-04-02 16:23:00,190 Parameters: domain_dimensions         = [16 16 16]
yt : [INFO     ] 2025-04-02 16:23:00,193 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-04-02 16:23:00,194 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-04-02 16:23:00,263 Parameters: current_time              = 2000.0
yt : [INFO     ] 2025-04-02 16:23:00,264 Parameters: domain_dimensions         = [16 16 16]
yt : [INFO     ] 2025-04-02 16:23:00,265 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-04-02 16:23:00,266 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-04-02 16:23:01,043 Parameters: current_time              = 4000.0
yt : [INFO     ] 2025-04-02 16:23:01,044 Parameters: domain_dimensions         = [16 16 16]
yt : [INFO     ] 2025-04-02 16:23:01,045 Parameters: domain_left_edge          = [0. 0. 0.]


(200, 19, 16, 16, 16)


In [5]:
tau = 1.
lbda = 1./tau
lbda_bar = lbda/(1+0.5*lbda)
factor1 = 2.*lbda_bar - lbda_bar**2
factor2 = temperature/cs2*Norm

fnoise_mode1_corv = np.zeros((ncomp, nx, ny, nz))
for a in range(1, 3+1):
    for n in range(Nframes):
        fnoise_mode1_corv[a,:,:,:] = fnoise_mode1_corv[a,:,:,:] + fnoise_ndarray[n,1,:,:,:]**2
    fnoise_mode1_corv[a,:,:,:] = fnoise_mode1_corv[a,:,:,:]/Nframes

fnoise_mode1_corv_normalized = fnoise_mode1_corv/temperature/factor1/0.5

a = 3
print(fnoise_mode1_corv_normalized[a,:,:,:])

print("numerical corvariance divided by theoretical value: mean = {:.5f} and var = {:.5f}"\
       .format(np.mean(fnoise_mode1_corv_normalized[a,:,:,:]), np.var(fnoise_mode1_corv_normalized[a,:,:,:])))


[[[0.82194946 1.11564626 0.89170306 ... 1.01135976 1.01869885 1.13543359]
  [1.03905282 1.08874514 1.18028189 ... 0.95023397 0.79739297 0.96743938]
  [0.774453   0.95856558 0.9524742  ... 0.9613499  0.84143927 0.97983932]
  ...
  [1.03300912 0.98751408 1.09948931 ... 0.82700739 1.1253309  0.92840994]
  [1.15204482 1.00313825 1.02924648 ... 0.94543352 1.13545502 0.90529411]
  [0.87452174 0.9978974  1.09078422 ... 1.07099899 1.03659272 1.0134142 ]]

 [[0.97214368 0.98208811 1.13188333 ... 0.82268126 0.85881115 0.90820015]
  [0.94964129 0.84088762 0.93785917 ... 1.06332661 0.94977743 1.0815712 ]
  [0.88851238 1.09474081 1.12214508 ... 1.13094517 1.14602048 1.06536737]
  ...
  [0.95121801 0.91644207 0.88433138 ... 0.90917411 1.09093237 1.25324914]
  [0.98073222 1.1017928  0.97794944 ... 0.98865004 1.11529257 1.00181264]
  [1.15550962 1.03607683 0.99596956 ... 1.02696604 0.96950765 1.02115806]]

 [[1.07910891 1.0466963  0.96943648 ... 1.10964727 0.8687676  1.05208388]
  [0.99031923 1.048679